In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
from datetime import date
from datetime import timedelta

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1)

###############################################################################################################################

data = []
def extract():
    driver.find_element_by_xpath("//span[text()='Expand All']").click()
    for k in range(len(driver.find_elements_by_tag_name('section'))):
        course_name = driver.find_elements_by_tag_name('section')[k].get_attribute('data-diffusion-coursename')

        if course_name:

            try:
                elem = driver.find_element_by_xpath(f"//section[{k+1}]/div[@class='rp-raceCourse__panel ui-accordion__content']/div[@class='rp-raceCourse__panel__detailsWrapper']").text.split('\n')
                for i in elem:
                    if 'Going: ' in i:
                        going = i.replace('Going: ', '')
            except:
                going = '-'

            for j in driver.find_elements_by_xpath(f"//section[{k+1}]/div[@class='rp-raceCourse__panel ui-accordion__content']/div[@class='rp-raceCourse__panel__container']"):
                winning_horse, race_time = '-', '-'
                Time = j.find_element_by_class_name('rp-raceCourse__panel__race__timePanel').text
                cnt = j.text.split('\n')
                for ie in range(len(cnt)):
                    if 'Going: ' in cnt[ie]:
                        winning_horse = cnt[ie+1].split('. ')[-1]
                    elif 'Time: ' in cnt[ie]:
                        race_time = cnt[ie].split('Time: ')[1].split('s ')[0]+'s'

                data.append([f'{date}', course_name, going, Time, winning_horse, race_time])
                                                      
#############################################################################################################################
                                            
today = date.today()
pages = [today - timedelta(days=i) for i in range(14)]

for p in pages:
    driver.get(f"https://www.racingpost.com/results/{p}")
    date = p
    try:
        extract()
    except:
        try:
            driver.find_elements_by_tag_name('img').send_keys(Keys.ESC)
            extract()
        except:
            print("There was some ERROR !")

#############################################################################################################################

SERVICE_ACCOUNT_FILE = 'keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

SAMPLE_SPREADSHEET_ID = '19gNgMJPeriXMJ2mRnBrulMO1apYnBBbBBGHZMhwVF2M'


try:
    service = build('sheets', 'v4', credentials=creds)
                                                      
    sheet = service.spreadsheets()
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="RACING!A2", valueInputOption="USER_ENTERED", body={"values":data}).execute()
                                                      
except HttpError as err:
    print(err)
    
driver.quit()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - There is no [win32] chromedriver for browser 99.0.4844 in cache
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\fagho\.wdm\drivers\chromedriver\win32\99.0.4844.51]
<ipython-input-2-19c6353c7059>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


There was some ERROR !
There was some ERROR !


WebDriverException: Message: chrome not reachable
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CA9943+2595139]
	Ordinal0 [0x00C3C9F1+2148849]
	Ordinal0 [0x00B343F0+1065968]
	Ordinal0 [0x00B287C2+1017794]
	Ordinal0 [0x00B28FF8+1019896]
	Ordinal0 [0x00B2A892+1026194]
	Ordinal0 [0x00B24219+999961]
	Ordinal0 [0x00B35860+1071200]
	Ordinal0 [0x00B8B2D2+1422034]
	Ordinal0 [0x00B7B806+1357830]
	Ordinal0 [0x00B56086+1204358]
	Ordinal0 [0x00B56F96+1208214]
	GetHandleVerifier [0x00E4B232+1658114]
	GetHandleVerifier [0x00F0312C+2411516]
	GetHandleVerifier [0x00D3F261+560433]
	GetHandleVerifier [0x00D3E366+556598]
	Ordinal0 [0x00C4286B+2173035]
	Ordinal0 [0x00C475F8+2192888]
	Ordinal0 [0x00C476E5+2193125]
	Ordinal0 [0x00C511FC+2232828]
	BaseThreadInitThunk [0x7581FA29+25]
	RtlGetAppContainerNamedObjectPath [0x778A7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x778A7A4E+238]


In [25]:
import os
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from google.oauth2 import service_account
# from Google import Create_Service

SERVICE_ACCOUNT_FILE = 'keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SAMPLE_SPREADSHEET_ID = '19gNgMJPeriXMJ2mRnBrulMO1apYnBBbBBGHZMhwVF2M'

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


service = build('sheets', 'v4', credentials=creds)
# service = Create_Service(SERVICE_ACCOUNT_FILE, 'drive', 'v4', SCOPES)


def convert_excel_file(file_path: str, folder_ids: list=None):
    if not os.path.exists(file_path):
        print(f"{file_path} not found")
        return
    
    try:
        file_metadata = {
            'name': os.path.splitext(os.path.basename(file_path))[0],
            'mimetype': 'application/vnd.google-apps.spreadsheet',
            'parents': folder_ids
        }
        
        media = MediaFileUpload(filename = file_path, mimetype = 'application/vnd.openxmlformats-officedocument.spreasheet.sheet')
        
        response = service.files().create(media_body=media, body=file_metadata).execute()
        
        print(response)
        return response
    
    except Exception as e:
        print(e)
        return
    
excel_files = os.listdir('./files')

for excel_file in excel_files:
    convert_excel_file(os.path.join('./files', excel_file))
    convert_excel_file(os.path.join('./files', excel_file), ['1Nu_m9sXc7V0-1PoEaQsw3LEmedkjUM40'])